In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

import torch
import kornia
import numpy as np
import gc
import os
import sys
%matplotlib notebook
import matplotlib.pyplot as plt

module_path = os.path.abspath(os.path.join('.'))
if module_path not in sys.path:
    sys.path.append(module_path)

from helpers import Trainer
from helpers import Logger
from helpers import Trial
from mibi_dataloader import MIBIData
from modules import Denoiser
from criteria import DenoiserLoss
from modules import Estimator
from criteria import EstimatorLoss
import utils

ImportError: numpy.core.multiarray failed to import

In [ ]:
# Load the data
main_dir = '/home/hazmat/GitHub/Denoiser/'
train_dir = main_dir + 'data/traindat/'
test_dir = main_dir + 'data/testdat/'

modl_dir = main_dir + 'models/'
rslt_dir = main_dir + 'results/'

train_ds = MIBIData(folder=train_dir, crop=255, scale=255, stride=8)
test_ds = MIBIData(folder=test_dir, crop=255, scale=255, stride=8)

In [ ]:
print()

In [ ]:
estimator_args = dict()
estimator_args['dims'] =    [8, 8, 8, 8, 8, 8, 8, 1]
estimator_args['kernels'] = [7, 7, 7, 7, 7, 7, 7, 7]

torch.cuda.empty_cache()
estimator = Estimator(**estimator_args)
estimator = Estimator.load_model(main_dir + 'estimator/models/estimator_saves/', 'model_10')

estimator.cuda()
estimator_logger = Logger(['loss'])
estimator_trainer = Trainer()

In [ ]:
# # Estimator training parameters
# estimator_train_args = dict()
# estimator_train_args['lr'] = 0.00033
# estimator_train_args['batch_size'] = 120
# estimator_train_args['epochs'] = 5
# estimator_train_args['report'] = 5
# estimator_train_args['crop'] = 255
# estimator_train_args['clip'] = 1
# estimator_train_args['decay'] = 0
# estimator_train_args['restart'] = False
# estimator_train_args['epoch_frac'] = 0.001
# # estimator_train_args['decay'] = 1e-5

# # Estimator loss parameters
# estimator_criterion = EstimatorLoss()

# train_ds.set_crop(estimator_train_args['crop'])

In [ ]:
# estimator.cuda()
# estimator_train_args['continue'] = True
# estimator_trainer.train(estimator, train_ds, estimator_criterion, estimator_logger, main_dir + 'estimator/models/', **estimator_train_args)
# print()

In [ ]:
lam = np.zeros((512,512))
for i in range(512):
    for j in range(512):
        lam[i,j] = (512-i)+(j)
lam = lam / 200
lam = np.expand_dims(lam,0)
lam = np.expand_dims(lam,0)
# lam = np.tile(lam,[100,1,1,1])
print(lam.shape)
x = torch.tensor(np.random.poisson(lam,lam.shape)).float().cuda()
lam = torch.tensor(lam).float().cuda()

In [ ]:
y = estimator.estimate(x)

In [ ]:
fig0 = plt.figure(figsize=(9,3))
_y = y*1.32
plt.subplot(1,3,1)
plt.imshow(lam[0,0,:,:].detach().cpu())
plt.subplot(1,3,2)
plt.imshow(_y[0,0,:,:].detach().cpu())
plt.subplot(1,3,3)
diff = torch.mean((lam-_y)**2)
plt.imshow(lam[0,0,:,:].detach().cpu() - _y[0,0,:,:].detach().cpu())
print(diff)

In [ ]:
from ipywidgets import *
from scipy.ndimage import gaussian_filter
from time import gmtime, strftime

# 12
img = train_ds.images[0]
img = img.unsqueeze(0).cuda()

fig1 = plt.figure(figsize=(10,5))
global img

l_hat = estimator.estimate(img)
l_hat = l_hat[0,0,:,:].detach().cpu()

plt.subplot(1,2,1)
plt.imshow(img[0,0,:,:].detach().cpu())
plt.subplot(1,2,2)
plt.imshow(l_hat)
print(l_hat.shape)

In [ ]:
print('hi')

In [ ]:
# Load the data
main_dir = '/home/hazmat/GitHub/Denoiser/'
train_dir = main_dir + 'data/traindat/'
test_dir = main_dir + 'data/testdat/'

modl_dir = main_dir + 'models/'
rslt_dir = main_dir + 'results/'

# train_ds = MIBIData(folder=train_dir, crop=81, scale=255, stride=8)
# test_ds = MIBIData(folder=test_dir, crop=81, scale=255, stride=8)

In [ ]:
denoiser_args = dict()
denoiser_args['dims'] =    [8, 16, 32, 8, 1]
denoiser_args['kernels'] = [3,  3,  3, 3, 3]

torch.cuda.empty_cache()
# denoiser = Denoiser(**denoiser_args)
denoiser = Denoiser.load_model(main_dir + 'denoiser/models/2020Mar04_08-57-20/', 'model_9')

denoiser.cuda()
denoiser_logger = Logger(['loss'])
denoiser_trainer = Trainer()

In [ ]:
# Denoiser training parameters
denoiser_train_args = dict()
denoiser_train_args['lr'] = 0.0005
denoiser_train_args['batch_size'] = 100
denoiser_train_args['epochs'] = 100
denoiser_train_args['report'] = 5
denoiser_train_args['crop'] = 81
denoiser_train_args['clip'] = 1
denoiser_train_args['decay'] = 0
denoiser_train_args['restart'] = False
denoiser_train_args['epoch_frac'] = 0.01
# denoiser_train_args['decay'] = 1e-5

# Denoiser loss parameters
denoiser_criterion = DenoiserLoss()

train_ds.set_crop(denoiser_train_args['crop'])

In [ ]:
denoiser.cuda()
denoiser_train_args['continue'] = False
denoiser_trainer.train(denoiser, train_ds, denoiser_criterion, denoiser_logger, main_dir + 'denoiser/models/', **denoiser_train_args)
print()

In [ ]:
# Denoiser training parameters
denoiser_train_args = dict()
denoiser_train_args['lr'] = 0.0005
denoiser_train_args['batch_size'] = 100
denoiser_train_args['epochs'] = 100
denoiser_train_args['report'] = 5
denoiser_train_args['crop'] = 81
denoiser_train_args['clip'] = 1
denoiser_train_args['decay'] = 0
denoiser_train_args['restart'] = False
denoiser_train_args['epoch_frac'] = 1
# denoiser_train_args['decay'] = 1e-5

# Denoiser loss parameters
denoiser_criterion = DenoiserLoss()

train_ds.set_crop(denoiser_train_args['crop'])

In [ ]:
denoiser.cuda()
denoiser_train_args['continue'] = False
denoiser_trainer.train(denoiser, train_ds, denoiser_criterion, denoiser_logger, main_dir + 'denoiser/models/', **denoiser_train_args)
print()

In [ ]:
lam = np.zeros((512,512))
for i in range(512):
    for j in range(512):
        lam[i,j] = (512-i)+(j)
lam = lam / 1000
lam = np.expand_dims(lam,0)
lam = np.expand_dims(lam,0)
# lam = np.tile(lam,[100,1,1,1])
print(lam.shape)
x = torch.tensor(np.random.poisson(lam,lam.shape)).float().cuda()
lam = torch.tensor(lam).float().cuda()

In [ ]:
y = estimator.estimate(x)
z = denoiser.denoise(x, y*1.45)

In [ ]:
y_scales = np.linspace(1.45,1.6,20)
diffs = list()

for y_scale in y_scales:
    y = estimator.estimate(x)
    z = denoiser.denoise(x, y*y_scale)
    # tuning_fig = plt.figure(figsize=(8,4))
    # plt.subplot(1,2,1)
    # plt.imshow(x[0,0,:,:].detach().cpu())
    # plt.subplot(1,2,2)
    # plt.imshow(z[0,0,:,:].detach().cpu())
    diff = torch.mean(z**2)
    print(diff)
    diffs.append(diff.detach().cpu())

In [ ]:
tuning_fig = plt.figure()
plt.plot(y_scales, diffs)

In [ ]:
# Load the data
leeat_ds = MIBIData(folder='/home/hazmat/GitHub/Denoiser/data/Point15/TIFs/', crop=81, scale=255, stride=8)

In [ ]:
from ipywidgets import *
from scipy.ndimage import gaussian_filter
from time import gmtime, strftime

img = leeat_ds.images[0]
img = img.unsqueeze(0)

fig1 = plt.figure(figsize=(14,7))
global img

def update(lam=0, cmax=1):
    global img
    x_lam = torch.zeros(img.shape)+lam
    print(x_lam.shape)
    dimg = denoiser.denoise(img.cuda(), x_lam)
    
    plt.subplot(1,2,1)
    kimg = img[0,0,:,:].cpu().detach()
    kimg = kimg[10:-10:1, 10:-10:1]
    plt.imshow(kimg, vmax=cmax)
    plt.axis('off')
    plt.title('Original Image (CD14)')

    plt.subplot(1,2,2)
    dimg = dimg[0,0,:,:].cpu().detach()
    dimg = dimg[10:-10:1, 10:-10:1]
    plt.imshow(dimg, vmax=cmax)
    plt.axis('off')
    plt.title('Denoised Image (CD14)')
    
#     plt.subplot(1,3,3)
#     plt.imshow(kimg-dimg)
#     plt.axis('off')
#     plt.title('Difference')
    
    print(strftime('%H:%M:%S', gmtime()))
    print(torch.mean(kimg-dimg))
    
interact(update, lam=widgets.FloatSlider(value=0.14, min=-0.1, max=100, step=0.01), cmax=widgets.IntSlider(value=3, min=0, max=20, step=1));

In [ ]:
from ipywidgets import *
from scipy.ndimage import gaussian_filter
from time import gmtime, strftime

img = test_ds.images[218]
img = img.unsqueeze(0)

fig1 = plt.figure(figsize=(14,7))
global img

def update(lam=0, cmax=1):
    global img
    x_lam = torch.zeros(img.shape)+lam
    print(x_lam.shape)
    dimg = denoiser.denoise(img.cuda(), x_lam)
    
    plt.subplot(1,2,1)
    kimg = img[0,0,:,:].cpu().detach()
    kimg = kimg[10:-10:1, 10:-10:1]
    plt.imshow(kimg, vmax=cmax)
    plt.axis('off')
    plt.title('Original Image (CD14)')

    plt.subplot(1,2,2)
    dimg = dimg[0,0,:,:].cpu().detach()
    dimg = dimg[10:-10:1, 10:-10:1]
    plt.imshow(dimg, vmax=cmax)
    plt.axis('off')
    plt.title('Denoised Image (CD14)')
    
#     plt.subplot(1,3,3)
#     plt.imshow(kimg-dimg)
#     plt.axis('off')
#     plt.title('Difference')
    
    print(strftime('%H:%M:%S', gmtime()))
    print(torch.mean(kimg-dimg))
    
interact(update, lam=widgets.FloatSlider(value=0.14, min=-0.1, max=1, step=0.01), cmax=widgets.IntSlider(value=3, min=0, max=20, step=1));

In [ ]:
from ipywidgets import *
from scipy.ndimage import gaussian_filter
from time import gmtime, strftime



# 12
img = train_ds.images[0]
img = img.unsqueeze(0)

fig1 = plt.figure(figsize=(18,6))
global img

def update(kind=False, lam=0, cmax=1):
    global img
    
    if not kind:
        x_lam = torch.zeros(img.shape)+lam
    else:
        x_lam = gaussian_filter(img[0,0,:,:], sigma=200)
        x_lam = torch.tensor(x_lam).unsqueeze(0).unsqueeze(0)*lam
    
    dimg = denoiser.denoise(img.cuda(), x_lam)
    
    plt.subplot(1,3,1)
    plt.imshow(x_lam[0,0,:,:], vmin=0)
    plt.axis('off')
    plt.title('Lambda')
    
    plt.subplot(1,3,2)
    kimg = img[0,0,:,:].cpu().detach()
    kimg = kimg[10:-10:1, 10:-10:1]
    plt.imshow(kimg, vmax=cmax)
    plt.axis('off')
    plt.title('Original Image (MAP2)')

    plt.subplot(1,3,3)
    dimg = dimg[0,0,:,:].cpu().detach()
    dimg = dimg[10:-10:1, 10:-10:1]
    plt.imshow(dimg, vmax=cmax)
    plt.axis('off')
    plt.title('Denoised Image (MAP2)')
    
    
    print(strftime('%H:%M:%S', gmtime()))
    print(torch.mean(kimg-dimg))
    
interact(update, kind=widgets.ToggleButton(value=True), lam=widgets.FloatSlider(value=1, min=0, max=1, step=0.01), cmax=widgets.IntSlider(value=3, min=0, max=20, step=1));